In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import os



In [59]:
from keras.models  import Sequential

from keras.layers import LSTM, Conv1D, Embedding, GlobalMaxPooling1D, Dense
from keras.models import Sequential, Model ## functional api를 쓰려면 model을 import하면 됨
from keras.layers import Dense, LSTM, Embedding ## 연산기능 
from keras.layers import Input, concatenate ## 보조적인 연산 기능
from keras.utils import plot_model

In [161]:
data=pd.read_csv("fuku_data")

/anaconda3/envs/TensorFlow/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (2,3,4,5,6,7,8,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,25

In [329]:
#data.head()

In [330]:
#data.tail()

In [165]:
colum=data.iloc[135792,0:9]
colum

Unnamed: 0               Date
Unnamed: 1               Time
Unnamed: 2    MP-1(microSv/h)
Unnamed: 3    MP-2(microSv/h)
Unnamed: 4    MP-3(microSv/h)
Unnamed: 5    MP-4(microSv/h)
Unnamed: 6    MP-5(microSv/h)
Unnamed: 7    MP-6(microSv/h)
Unnamed: 8    MP-7(microSv/h)
Name: 135792, dtype: object

In [166]:
data=data.iloc[:,0:9]

In [331]:
#data.head()

In [168]:
data.columns=[i for i in colum]

In [170]:
data.head()
data.tail()

,Date,Time,MP-1(microSv/h),MP-2(microSv/h),MP-3(microSv/h),MP-4(microSv/h),MP-5(microSv/h),MP-6(microSv/h),MP-7(microSv/h)
135788,2019/08/10,23:20,0.195,0.173,0.297,0.279,0.273,0.147,0.092
135789,2019/08/10,23:30,0.192,0.173,0.297,0.278,0.271,0.148,0.091
135790,2019/08/10,23:40,0.191,0.173,0.300,0.281,0.270,0.148,0.090
135791,2019/08/10,23:50,0.194,0.174,0.299,0.282,0.274,0.148,0.091
135792,Date,Time,MP-1(microSv/h),MP-2(microSv/h),MP-3(microSv/h),MP-4(microSv/h),MP-5(microSv/h),MP-6(microSv/h),MP-7(microSv/h)


In [296]:
data=data.iloc[:-1,:]
data.tail()

,Date,Time,MP-1(microSv/h),MP-2(microSv/h),MP-3(microSv/h),MP-4(microSv/h),MP-5(microSv/h),MP-6(microSv/h),MP-7(microSv/h)
135784,2019/08/10,22:40,0.194,0.175,0.295,0.280,0.273,0.147,0.091
135785,2019/08/10,22:50,0.193,0.173,0.296,0.276,0.271,0.148,0.091
135786,2019/08/10,23:00,0.192,0.172,0.300,0.282,0.270,0.148,0.090
135787,2019/08/10,23:10,0.194,0.174,0.297,0.281,0.274,0.147,0.092
135788,2019/08/10,23:20,0.195,0.173,0.297,0.279,0.273,0.147,0.092


# Create the Window

In [325]:
mp1=data["MP-1(microSv/h)"].values
mp1=np.array(mp1)

check=[]
for i in mp1:
    if i=="-":
        check.append(float(i))

mp1_new=np.delete(mp1,check)
     

# count = data["MP-1(microSv/h)"].value_counts() 
# print(count)

# sum(mp1=="-")

# for i in mp1:
#     if i=="-":
#         check.append(i)

ValueError: could not convert string to float: '-'

In [315]:
seq_len=100
seq_length=seq_len+5

result=[] # total window

for i in range(len(mp1)-seq_length):
    result.append(mp1[i:i+seq_length])


In [316]:
result=np.array(result)
result.shape

(135684, 105)

# Normalization

# Split the Test and Train

In [317]:
row=int(round(result.shape[0]*0.9))
train=result[0:row,:]
np.random.shuffle(train)

In [318]:
x_train=train[:,:-5]
y_train=train[:,-5:]
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
#y_train=np.reshape(y_train,(y_train.shape[0],y_train.shape[1],1))

x_test=result[row:,:-5]
y_test=result[row:,-5:]
x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
#y_test=np.reshape(y_test,(y_test.shape[0],y_test.shape[1],1))


print(x_train.shape,x_test.shape)
print(y_train.shape,y_test.shape)

(122116, 100, 1) (13568, 100, 1)
(122116, 5) (13568, 5)


# build a model

In [319]:
model=Sequential()

model.add(LSTM(100, return_sequences=True, input_shape=(100,1)))
model.add(LSTM(64,return_sequences=True))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(16,return_sequences=False))
model.add(Dense(5,activation="linear"))
model.compile(loss="mse",optimizer="rmsprop")
              
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_53 (LSTM)               (None, 100, 100)          40800     
_________________________________________________________________
lstm_54 (LSTM)               (None, 100, 64)           42240     
_________________________________________________________________
lstm_55 (LSTM)               (None, 100, 32)           12416     
_________________________________________________________________
lstm_56 (LSTM)               (None, 16)                3136      
_________________________________________________________________
dense_8 (Dense)              (None, 5)                 85        
Total params: 98,677
Trainable params: 98,677
Non-trainable params: 0
_________________________________________________________________


# Training

In [320]:

model.fit(x_train,y_train,
         validation_data=(x_test,y_test),
          batch_size=50,
          epochs=20
         )

Train on 122116 samples, validate on 13568 samples
Epoch 1/20


ValueError: could not convert string to float: '-'

# Prediction

In [328]:
pred=model.predict(x_test)

fig=plt.figure(facecolor="white")
ax=fig.add_subplot(11)
ax.plot(y_test,label="Ture")
ax.plot(pred,label="Prediction")
ax.lengend()
plt.show()

ValueError: Integer subplot specification must be a three-digit number, not 11

<Figure size 432x288 with 0 Axes>